In [14]:
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
import numpy as np
from glob import glob
import random
from tqdm import tqdm
from tensorflow import keras
from keras.preprocessing import image
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from PIL import ImageFile
import matplotlib.pyplot as plt
from skimage.transform import resize
import os
import zipfile
import pandas as pd
import pickle
import shutil

In [19]:
def my_alternative_validation_split_method(main_dir_path, split=0.2):
    
    shutil.copytree(main_dir_path, main_dir_path + '_alt')
    
    main_dir_path = main_dir_path + '_alt'
    train_path = main_dir_path + '/train'
    validation_path = main_dir_path + '/validate'
    
    if os.path.isdir(train_path) or os.path.isdir(validation_path):
        print('You have already split your training and test data')
        
    else:
        sub_dir_names = os.listdir(main_dir_path)
        os.mkdir(train_path)
        os.mkdir(validation_path)
        for sub_dir_name in sub_dir_names:
            name = sub_dir_name.split('_')[0]
            os.mkdir(train_path + '/'+ name)
            os.mkdir(validation_path + '/'+ name)
            images = os.listdir(main_dir_path +'/' + sub_dir_name)
            val_n = int(split * len(images))
            val_images = images[:val_n]
            for i in range(len(val_images)):
                for filepath in glob(main_dir_path +'/' + sub_dir_name + '/' + val_images[i]):
                    shutil.move(main_dir_path +'/' + sub_dir_name + '/' + val_images[i], validation_path +'/' + name + '/' + val_images[i])
            training_images = os.listdir(main_dir_path +'/' + sub_dir_name)
            print(f'{name}: {len(images)} total images')
            print(f'{name}: {len(training_images)} training images')
            print(f'{name}: {val_n} validation images')
            print('validation split: ', 100 * round(val_n/(val_n + len(training_images)),2), '%')
            print()
            for i in range(len(training_images)):
                for filepath in glob(main_dir_path +'/' + sub_dir_name + '/' + training_images[i]):
                    shutil.move(main_dir_path +'/' + sub_dir_name + '/' + training_images[i], train_path +'/' + name + '/' + training_images[i])
        for sub_dir_name in sub_dir_names:
            os.rmdir(main_dir_path +'/' + sub_dir_name)

In [27]:
data = 'resources/data_for_training_06.zip'

zip_ref = zipfile.ZipFile(data, 'r')
zip_ref.extractall('resources')
zip_ref.close()

In [35]:
# main_dir = 'resources/data_for_training_06'

# if '.DS_Store/' in os.listdir(main_dir):
#    os.remove('.DS_Store/')    

In [36]:
my_alternative_validation_split_method(main_dir, split=0.2)

brick: 1872 total images
brick: 1498 training images
brick: 374 validation images
validation split:  20.0 %

siding: 1529 total images
siding: 1224 training images
siding: 305 validation images
validation split:  20.0 %

unknown: 704 total images
unknown: 564 training images
unknown: 140 validation images
validation split:  20.0 %



In [ ]:
model = Sequential()
# ADJUST FILTER?
model.add(Conv2D(filters=2, kernel_size=2, padding='same',
                 activation='relu', input_shape=(500, 500, 3)))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=4, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.1))

model.add(Conv2D(filters=8, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=12, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))

model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.4))


model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(3, activation='softmax'))


model.summary()

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# # Adding rescale, rotation_range, width_shift_range, height_shift_range,
# # shear_range, zoom_range, and horizontal flip to our ImageDataGenerator
# train_datagen = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=40,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,)

# # Note that the validation data should not be augmented!
# val_datagen = ImageDataGenerator(rescale=1./255)

# # Flow training images in batches of 32 using train_datagen generator
# train_generator = train_datagen.flow_from_directory(
#         main_dir + '_alt' + '/train',  # This is the source directory for training images
#         target_size=(400, 400),  
#         batch_size=20,
#         # Since we use binary_crossentropy loss, we need binary labels
#         class_mode='categorical')

# # Flow validation images in batches of 32 using val_datagen generator
# validation_generator = val_datagen.flow_from_directory(
#         main_dir + '_alt' + '/validate',
#         target_size=(400, 400),
#         batch_size=20,
#         class_mode='categorical')

In [5]:
# model = Sequential()

# model.add(Conv2D(filters=4, kernel_size=2, padding='same',
#                  activation='relu', input_shape=(400, 400, 3)))
# model.add(MaxPooling2D(pool_size=2))

# model.add(Conv2D(filters=8, kernel_size=2, padding='same', activation='relu'))
# model.add(MaxPooling2D(pool_size=2))
# model.add(Dropout(0.1))

# model.add(Conv2D(filters=12, kernel_size=2, padding='same', activation='relu'))
# model.add(MaxPooling2D(pool_size=2))
# model.add(Dropout(0.2))

# model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu'))
# model.add(MaxPooling2D(pool_size=2))
# model.add(Dropout(0.3))



# model.add(Flatten())

# model.add(Dense(256, activation='relu'))

# model.add(Dropout(0.4))

# model.add(Dense(3, activation='softmax'))


# model.summary()

# model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# epochs = 15

# checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.image_classifier.hdf5',
#                                verbose=1, save_best_only=True)

# history = model.fit(train_tensors, train_targets, validation_data=(valid_tensors, valid_targets),
#           epochs=epochs, batch_size=64, callbacks=[checkpointer], verbose=1)